# BERT Probing

Il processo implementato segue la metodologia illustrata dal dottor Alessio Miaschi nel seguente repository: https://github.com/gsarti/lcl23-xnlm-lab/blob/main/notebooks/1.2_Probing.ipynb

In [ ]:
# Specifica il percorso della cartella dove è presente il set 
sentences_file = "/content/drive/My Drive/TMAG_Probing_Cita/dfs_post-cleaning/UD_15000records/Italian_UD.tsv"
# Lettura del DataFrame da un file TSV
df = pd.read_csv(sentences_file, sep='\t', quoting=3)

# Specifica il percorso della cartella dove è presente il modello finetunato
model_folder_path = "/content/drive/My Drive/TMAG_Probing_Cita/FT_Models/lenght150/FTmode_2ep_set3_best"
##### VARIAZIONE IN PROBING SU MODELLO PRETRAINED
##### model_name = "dbmdz/bert-base-italian-xxl-uncased"

# Carica il tokenizer 
tokenizer = BertTokenizer.from_pretrained(model_folder_path)
##### VARIAZIONE IN PROBING SU MODELLO PRETRAINED
##### tokenizer = AutoTokenizer.from_pretrained(model_name)

# Carica il modello
model = BertForSequenceClassification.from_pretrained(model_folder_path, output_hidden_states=True)
##### VARIAZIONE IN PROBING SU MODELLO PRETRAINED
##### model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

# Trasferisce il modello sulla GPU se disponibile, altrimenti rimane su CPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Liste vuote per memorizzare gli embedding, gli ID e le frasi
embeddings = []
ids = []
sentences = []

# Apertura del file contenente le frasi da elaborare
with open(sentences_file, 'r') as f:
    
    # Iterazione attraverso ciascuna riga nel file
    for line in f:
        # Lista per memorizzare le rappresentazioni della frase corrente
        embFrase=[]
        # Estrazione degli elementi dalla riga separati da tabulazioni
        elements = line.rstrip('\n').split('\t')
        sent_id = elements[0]
        sentence = elements[1]
        # Tokenizzazione della frase utilizzando il tokenizer
        input_ids = tokenizer.encode(sentence, add_special_tokens=True)
        input_ids = torch.tensor([input_ids]).to(device)

        # Estrazione delle rappresentazioni 
        with torch.no_grad():
            outputs = model(input_ids)
            # Estrazione delle rappresentazioni dal primo layer
            rappresentazione_layer6 = torch.squeeze(outputs['hidden_states'][1][:, 0, :])
            # Estrazione delle rappresentazioni dall'ultimo layer
            rappresentazione_layer12 = torch.squeeze(outputs['hidden_states'][-1][:,0,:])
            # Crea una lista contenente i tensori dei layer 6 e 12
            embFrase = [rappresentazione_layer6, rappresentazione_layer12]
            embeddings.append(embFrase)
            # Aggiunta dell'ID e della frase alle rispettive liste
            ids.append(sent_id)
            sentences.append(sentence)

# Salvataggio delle rappresentazioni
#with open("/content/drive/My Drive/TMAG_Probing_Cita/Bert_Representations/lenght150/Finetuned/ ItalianUD_Representations150_pt.pkl", "wb") as fOut:
  #pickle.dump({'ids': ids, 'sentences': sentences, 'embeddings': embeddings}, fOut)
##### VARIAZIONE IN PROBING SU MODELLO PRETRAINED:
#with open("/content/drive/My Drive/TMAG_Probing_Cita/Bert_Representations/lenght150/Pretrained/ ItalianUD_Representations150_pt.pkl", "wb") as fOut:
  #pickle.dump({'ids': ids, 'sentences': sentences, 'embeddings': embeddings}, fOut)
    
# Specifica il percorso della cartella dove sono presenti le rappresentazioni create tramite modello  
file_path = "/content/drive/My Drive/TMAG_Probing_Cita/Bert_Representations/lenght150/Finetuned/ ItalianUD_Representations150_ft.pkl"
##### VARIAZIONE IN PROBING SU MODELLO PRETRAINED:
##### file_path = "/content/drive/My Drive/TMAG_Probing_Cita/Bert_Representations/lenght150/Pretrained/ ItalianUD_Representations150_pt.pkl"

# Apre il file in modalità lettura 
with open(file_path, "rb") as fIn:
# Carica i dati dal file
data = pickle.load(fIn)

# Definizione delle variabili per accedere ai dati nel dizionario
ids = data['ids']
sentences = data['sentences']
embeddings = data['embeddings']

# Caricamento del dataset con le features linguistiche estratte tramite ProfilingUD
Features_file = "/content/drive/My Drive/TMAG_Probing_Cita/dfs_post-cleaning/UD_15000records/Gold_UD.tsv"
# Letture del DataFrame da un file .tsv
df = pd.read_csv(Features_file, delimiter='\t')
df = df.set_index("ID")

# Salva in una lista i nomi di tutte le features
lista_features = df.columns.tolist()

# Funzione che data la lista delle features restituisce diverse liste per ogni macro-gruppo di features
def create_feature_lists(lista_colonne):
    
    # Definizione delle liste vuote
    raw_text_properties = []
    lexical_variety = []
    morphosyntactic_info_1 = []
    morphosyntactic_info_2_infmorph =[]
    verbal_predicate_structure = []
    parsed_tree_structures = []
    order_of_elements = []
    syntactic_relations = []
    subordination_features = []
    all_features = []
 
    # Inserimento di ogni feature all'interno della lista dedicata
    for colonna in lista_colonne:
        if 'n_sentences' in colonna or 'n_tokens' in colonna or 'tokens_per_sent' in colonna or 'char_per_tok' in colonna:
            raw_text_properties.append(colonna)
        elif 'ttr_' in colonna or 'lexical_density' in colonna:
            lexical_variety.append(colonna)
        elif 'upos_dist_' in colonna or 'lexical_density' in colonna:
            morphosyntactic_info_1.append(colonna)
        elif 'verbs_tense_dist_' in colonna or 'verbs_mood_dist_' in colonna or 'verbs_form_dist_' in colonna or 'verbs_num_pers_dist_' in colonna:
            morphosyntactic_info_2_infmorph.append(colonna)
        elif 'verbal_head_per_sent' in colonna or 'verbal_root_perc' in colonna or 'avg_verb_edges' in colonna or 'verb_edges_dist_' in colonna:
            verbal_predicate_structure.append(colonna)
        elif 'avg_max_depth' in colonna or 'avg_token_per_clause' in colonna or 'avg_links_len' in colonna or 'avg_max_links_len' in colonna or 'max_links_len' in colonna:
            parsed_tree_structures.append(colonna)
        elif 'obj_pre' in colonna or 'obj_post' in colonna or 'subj_pre' in colonna or 'subj_post' in colonna:
            order_of_elements.append(colonna)
        elif 'dep_dist_' in colonna:
            syntactic_relations.append(colonna)
        elif 'principal_proposition_dist' in colonna or 'subordinate_proposition_dist' in colonna or 'subordinate_post' in colonna or 'subordinate_pre' in colonna or 'avg_subordinate_chain_len' in colonna or 'subordinate_dist_' in colonna:
            subordination_features.append(colonna)
        all_features.append(colonna)
    return {
        "Proprietà del Testo Grezzo": raw_text_properties,
        "Variazione Lessicale": lexical_variety,
        "Informazioni Morfosintattiche": morphosyntactic_info_1,
        "Informazioni Morfosintattiche - Morfologia flessiva": morphosyntactic_info_2_infmorph,
        "Caratteristiche Sintattiche - Struttura del Predicato Verbale": verbal_predicate_structure,
        "Caratteristiche Sintattiche - Strutture Alberate Analizzate Globalmente e Localmente": parsed_tree_structures,
        "Caratteristiche Sintattiche - Ordine degli Elementi": order_of_elements,
        "Caratteristiche Sintattiche - Relazioni Sintattiche": syntactic_relations,
        "Caratteristiche Sintattiche - Uso della Subordinazione": subordination_features,
        "Tutte le Features": all_features
    }
# Chiama la funzione create_feature_lists sulla lista delle features
features_divise = create_feature_lists(lista_features)

# Funzione per esecuzione del processo di probing
def probing(embeddings, df, ids, feature, layer):

    # Dataset contenente solo le feature linguistiche selezionate 
    df_feature = df[feature]
    X = []
    y = []
    # Itera gli id e gli embeddings
    for id, sentence in zip(ids, embeddings):
        # Estrae le rappresentazioni dal token [CLS] (primo token in ogni sequenza di input)
        cls_embedding = sentence[layer].tolist()
        # Accesso al valore della feature linguistica per la data frase basata sull'id 
        feat = df_feature.loc[id]
        X.append(cls_embedding)
        y.append(feat)

    # Divide il dataset in train e test 
    X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(X, y, ids, test_size=0.20, random_state=42)

    # Definizione del probing model
    probing_model = LinearSVR(dual=False, loss='squared_epsilon_insensitive')

    # Addestramento del modello sul training set e poi effettua predizioni sul test
    probing_model.fit(X_train, y_train)
    y_pred = probing_model.predict(X_test)

    # Valutazione delle predizioni
    corr, p_val = spearmanr(y_test, y_pred)

    # Salvataggio delle predizioni per ulteriori analisi 
    df_preds = pd.DataFrame(columns=["sent_id", "y_true", "y_pred"])
    df_preds["sent_id"] = ids_test
    df_preds["y_true"] = y_test
    df_preds["y_pred"] = y_pred
    df_preds.to_csv(f'/content/drive/My Drive/TMAG_Probing_Cita/Results_SVReg_allMod/lenght150/Result_SVReg_Finetuned/ results_<@\{layer\}@>_<@\{feature\}@>.tsv',
                  sep='\t', index=True)
    return corr, p_val

# Definizione ed addestramento del modello baseline
def baseline(df, feature):
    
    # Seleziona "n_tokens" come X per il modello probe baseline
    X = df["n_tokens"].to_numpy()
    X = X.reshape(-1, 1)
    y = df[feature].to_numpy()
    # Divide dataset in train e test 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    # Definizione modello probe
    probing_model = LinearSVR(dual=False, loss='squared_epsilon_insensitive')
    # Fit del modello sui dati di addestramento e predizione su quelli di test 
    probing_model.fit(X_train, y_train)
    y_pred = probing_model.predict(X_test)

    # Valutazione delle previsioni
    corr, p_val = spearmanr(y_test, y_pred)
    return corr, p_val


# Stampa delle liste generate
for categoria, features in features_divise.items():
    print(f"{categoria}:", features)
    
# Itera sulle liste generate dalla funzione create_feature_lists
for categoria, selected_features in features_divise.items():
    
    # Lista con gli indici dei tensori dei layer interessati
    layers = [1, 12]
    results = pd.DataFrame(index=selected_features, columns=layers + ["Baseline"])
    # Itera sulle feature selezionate
    for feature in selected_features:
        # Itera sui layer
        for layer in layers:
            # Chiamata alla funzione di probing
            corr, p_val = probing(embeddings, df, ids, feature, layer)
            # Salva il punteggio solo se la correlazione è statisticamente significativa (valore p < 0.05)
            if p_val < 0.05:
                results.loc[feature][layer] = corr

        # Modello baseline addestrato a prevedere le features selezionate usando solo "n_tokens" come feature di input
        corr_baseline, p_val_baseline = baseline(df, feature)
        if p_val_baseline < 0.05:
            results.loc[feature]["Baseline"] = corr_baseline

    # Stampa i risultati per la categoria corrente
    print(f"Risultati per la categoria {categoria}:\n{results}")

    # Formattazione del dataframe per la visualizzazione 
    results = results[layers]
    results = results.T
    results["Layers"] = results.index

    # Visualizzazione delle performance tramite plot 
    fig, ax = plt.subplots(figsize=(8, 6))
    g = sns.lineplot(data=results.melt(["Layers"]), x="Layers", y="value", hue="variable", linewidth=2)
    handles, labels = g.get_legend_handles_labels()
    g.legend(handles=handles, labels=labels, bbox_to_anchor=(0., 1.02, 1., .102),
             loc='lower left', ncol=3, mode="expand")
    # Imposta i ticks e le etichette sull'asse x
    ax.set_xticks([0, 1])
    ax.set_xticklabels([1, 12])
    
    # Mostra il grafico
    plt.show()